# 프로그램 소개
테스트
테스트

In [43]:
%pip install openpyxl
%pip install yt-dlp
%pip install colorama
# !sudo apt-get install ffmpeg

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# 여기 아래줄의 ""안에 있는 이름을 다운로드한 엑셀 파일의 이름으로 바꾸세요
excel_file_name = "2024년 6월 보컬로이드 소모임 플레이리스트"

from playlist_generator import playlist

playlist.generate(excel_file_name)


성공: 2024년 6월 플레이리스트를 생성합니다.


In [44]:
# 여기 아래줄의 ""안에 있는 이름을 다운로드한 엑셀 파일의 이름으로 바꾸세요
file_name = "2024년 6월 보컬로이드 소모임 플레이리스트"

# 그 외 설정들
date_year_cell = "I4"
date_month_cell = "J4"

start_row = 3

excel_key_names = [
    "name",
    "nickname",
    "originalLink",
    "downloadableLink",
    "songName",
    "songNameKor",
    "composer",
    "comment",
    "korLyricsLink",
]

additional_key_names = [
    "fileName",
    "audioDownloaded",
    "thumbnailDownloaded",
]

final_output_key_names = excel_key_names[1:] + additional_key_names[0]


audio_folder_name = "audio"
thumbnail_folder_name = "thumbnail"

from IPython.display import display_markdown
from colorama import Fore, Style
import openpyxl as xl
import datetime
import yt_dlp
import os
import playlist_generator as plygen

excel_path  = f"{file_name}.xlsx"
workbook = xl.load_workbook(excel_path)
sheets = workbook.worksheets
date_sheet = sheets[0]
recommend_sheet = sheets[1]

try:
    date_year = int(date_sheet[date_year_cell].value)
    date_month = int(date_sheet[date_month_cell].value)
    date = datetime.date(date_year, date_month, 1)
    display_markdown(f"{date.year}년 {date.month}월달 플레이리스트를 생성합니다", raw=True)
except ValueError:
    display_markdown(f"""제대로 플레이리스트를 생성하지 못했습니다. 
                     {workbook.sheetnames[0]}에서 {date_year_cell}과 {date_month_cell}에 
                     각각 연도와 월을 제대로 기입했는지 확인해주세요.""", 
                     raw=True)
    exit(0)


data_array = []
anon_idx = ord("A")
row_idx = start_row
replace_str = "\\/:*?\"<>|"
file_name_key = additional_key_names[0] # 파일 이름에 해당하는 키 이름
essential_key_names = excel_key_names[2:6] # 필수로 채워져야 하는 키 이름

display_markdown("엑셀 파일에서 정보를 읽어오는 중입니다", raw=True)

try:
    while (recommend_sheet.cell(row=row_idx, column=1).value is not None):
        data = {}
        name = None
        for idx, key in enumerate(excel_key_names):
            cell = recommend_sheet.cell(row=row_idx, column=idx + 1)
            if (key == "name"):
                display_markdown(f"**{cell.value}**님의 데이터를 처리중입니다", raw=True)
                name = cell.value
            
            if key in essential_key_names and cell.value is None:
                key_name_excel = recommend_sheet.cell(row=2, column=idx + 1).value
                raise Exception(name, key_name_excel)

            # 닉네임이 없을 경우 익명으로 처리
            if (key == "nickname") and cell.value is None:
                data[key] = "익명" + chr(anon_idx)
                anon_idx += 1
            # 파일명에 사용할 수 없는 문자 변경
            elif (key == "songNameKor"):
                file_name = cell.value
                for char in replace_str:
                    file_name = file_name.replace(char, "_")
                data[key] = cell.value
                data[file_name_key] = file_name
            else:
                data[key] = cell.value
        data_array.append(data)
        row_idx += 1
except Exception as e:
    name, key_name_excel = e.args
    display_markdown(f"**{name}**의 **{key_name_excel}**에 해당하는 정보가 없는 것 같습니다. 엑셀 파일을 확인해주세요.", raw=True)
    exit(0)


audio_ydl_opts = {
            "format": "bestaudio/best",
            "postprocessors": [
                {
                    "key": "FFmpegExtractAudio",
                    "preferredcodec": "mp3",
                    "preferredquality": "192",
                }
            ],
            "quiet": True,
        }

if not os.path.exists(audio_folder_name):
    os.mkdir(audio_folder_name)

existing_audio_files = os.listdir(audio_folder_name)
for file in existing_audio_files:
    os.remove(os.path.join(audio_folder_name, file))

link_key_name = excel_key_names[3]

for data in data_array:
    try:
        ydl_opts = audio_ydl_opts.copy()
        ydl_opts["outtmpl"] = os.path.join(audio_folder_name, data[file_name_key] + ".%(ext)s")
        link = data[link_key_name]
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download(link)
        print(f"{Fore.BLUE}{data["songNameKor"]} {Style.RESET_ALL}다운로드에 성공했습니다.")
    except yt_dlp.utils.DownloadError as e:
        print(f"다운로드에 실패했습니다. 영상을 스킵합니다.\n{e}")
    except Exception as e:
        print(f"에러가 발생하였습니다.\n{e}")

2024년 6월달 플레이리스트를 생성합니다

엑셀 파일에서 정보를 읽어오는 중입니다

**김영찬**님의 데이터를 처리중입니다

**김예린**님의 데이터를 처리중입니다

**문윤서**님의 데이터를 처리중입니다

...

**이재헌**님의 데이터를 처리중입니다

**임재현**님의 데이터를 처리중입니다

**한부현**님의 데이터를 처리중입니다

**한수진**님의 데이터를 처리중입니다

Fairytale, (VY1V4 cover) 다운로드에 성공했습니다.          
절취선 다운로드에 성공했습니다.                               
Calc. 다운로드에 성공했습니다.                             
남자답지 못해 다운로드에 성공했습니다.                           
미래는 어떨까 다운로드에 성공했습니다.                           
리얼라인 다운로드에 성공했습니다.                              
밤새도록 널 생각해(10th anniv) 다운로드에 성공했습니다.            
달빛 랩소디 다운로드에 성공했습니다.                            
모자이크 롤 (SeeU cover) 다운로드에 성공했습니다.               
where shall we go? 다운로드에 성공했습니다.                


In [4]:
# 여기 아래줄의 ""안에 있는 이름을 다운로드한 엑셀 파일의 이름으로 바꾸세요
file_name = "2024년 6월 보컬로이드 소모임 플레이리스트"

import openpyxl as xl
import subprocess
import yt_dlp
import json
import os

"""
파일 만들때 꼭 확인 해야 하는 부분!
"""
PATH_PREFIX = "2024-06"
EXCEL_FILE_NAME = "2024-06"
START_ROW = 10
COLUMN_LENGTH = 8
JSON_INFO = [
    "name",
    "nickname",
    "audioLink",
    "songName",
    "songNameKor",
    "composer",
    "comment",
    "lyricsKor",
]

FILE_NAME = "fileName"
IMG_PATH = "imgPath"

# 엑셀에서 데이터 읽어오기
excel_path = os.path.join(f"./{PATH_PREFIX}/{EXCEL_FILE_NAME}.xlsx")
excel_document = xl.load_workbook(excel_path)
sheets = excel_document.sheetnames
sheet = excel_document[sheets[0]]

# 엑셀에서 정보 추출, JSON_INFO에 맞게 데이터를 info_array에 저장
dl_link = []
data_array = []
anon_idx = ord("A")

for row in range(START_ROW, sheet.max_row + 1):
    if sheet.cell(row=row, column=1).value is None:
        break
    info = {}
    for col in range(1, COLUMN_LENGTH + 1):
        cell_value = sheet.cell(row=row, column=col).value
        # 닉네임이 없을 경우 익명으로 처리
        if col == JSON_INFO.index("nickname") + 1 and cell_value is None:
            cell_value = "익명" + chr(anon_idx)
            anon_idx += 1
        # 파일명에 사용할 수 없는 문자 제거
        elif col == JSON_INFO.index("songNameKor") + 1:
            replace_str = '\\/:*?"<>|'
            file_name = cell_value
            for c in replace_str:
                file_name = file_name.replace(c, "#")
            info[FILE_NAME] = file_name
        info[JSON_INFO[col - 1]] = cell_value

    data_array.append(info)
    dl_link.append(info["audioLink"])

audio_path = os.path.join(f"./{PATH_PREFIX}/audio")
thumbnail_path = os.path.join(f"./{PATH_PREFIX}/visual/thumbnail")

dl_audio = True
dl_thumbnail = False

# 노래 다운로드
for idx, link in enumerate(dl_link):
    if dl_audio:
        ydl_opts = {
            "format": "bestaudio/best",
            "postprocessors": [
                {
                    "key": "FFmpegExtractAudio",
                    "preferredcodec": "mp3",
                    "preferredquality": "192",
                }
            ],
            "outtmpl": os.path.join(
                audio_path, data_array[idx][FILE_NAME] + ".%(ext)s"
            ),
        }

        try:
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                ydl.download(link)
            print("다운로드에 성공했습니다.")
        except yt_dlp.utils.DownloadError as e:
            print(f"다운로드에 실패했습니다. 영상을 스킵합니다.\n{e}")
        except Exception as e:
            print(f"에러가 발생하였습니다.\n{e}")


# 썸네일 다운로드
for idx, link in enumerate(dl_link):
    if dl_thumbnail:
        thumbnail_opts = {
            "skip_download": True,  # Skip downloading the video
            "writethumbnail": True,  # Download the thumbnail
            "outtmpl": os.path.join(
                thumbnail_path, data_array[idx][FILE_NAME] + ".%(ext)s"
            ),
        }

        with yt_dlp.YoutubeDL(thumbnail_opts) as ydl:
            ydl.download(link)

        # get thumbnail file which starts with the song name
        thumbnail_files = os.listdir(thumbnail_path)
        thumbnail_files = [
            file
            for file in thumbnail_files
            if file.startswith(data_array[idx][FILE_NAME])
        ]
        thumbnail = thumbnail_files[0]

        if not thumbnail.endswith(".png"):
            subprocess.run(
                [
                    "ffmpeg",
                    "-i",
                    f"{thumbnail_path}/{thumbnail}",
                    f"{thumbnail_path}/{data_array[idx][FILE_NAME]}.png",
                ]
            )

            os.remove(f"{thumbnail_path}/{thumbnail}")

    data_array[idx][IMG_PATH] = str(
        os.path.abspath(
            os.path.join(thumbnail_path, f"{data_array[idx][FILE_NAME]}.png")
        )
    ).replace("\\", "/")


json_path = os.path.join(f"./{PATH_PREFIX}/{EXCEL_FILE_NAME}.json")
# json_path = os.path.abspath("./temp/data.json")
with open(
    json_path,
    "w",
    encoding="utf-8",
) as f:
    json.dump(data_array, f, ensure_ascii=False, indent=2)


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip
